In [19]:

import streamlit as st
import pandas as pd
import numpy as np
import requests
import spacy
import tqdm
from collections import Counter

import json

#search_query = "Digitalisierung"
search_query = "Klima"

json = {'search_query':search_query, 
        'dataset_name':'deu', 
        'start_date':'2018-1-1', 
        'end_date':'2019-1-1'}
result = requests.post('https://penelope.vub.be/parliament-data/get-speeches', json=json)
 

In [20]:
speeches = []
parties = []
for speech in result.json()["speeches"]:
    speeches.append(speech)
    parties.append(speech['party'])
    

In [21]:
len(speeches)
partycounts = dict(Counter(parties).most_common())
partycounts

{'CDU/CSU': 216,
 'SPD': 191,
 'BÜNDNIS 90/DIE GRÜNEN': 139,
 'AfD': 93,
 'DIE LINKE': 84,
 'FDP': 74,
 'BÜNDNIS\xa090/DIE GRÜNEN': 16,
 'fraktionslos': 9,
 'Fraktionslos': 2}

In [22]:
speeches[10]

{'date': '2019-03-20',
 'discussion_title': 'Tagesordnungspunkt 2 2019-03-20',
 'id': 'ID198804700',
 'name': 'Andreas Scheuer',
 'party': 'CDU/CSU',
 'period': '19',
 'text': 'Ich habe immer gesagt: Wir brauchen gute Mobilität und saubere Luft. Wir brauchen aber auch die Konzentration darauf, dass wir mit neuen Antriebstechnologien Ziele erreichen. Dazu zählen auch die alternativen Kraftstoffe. Ich habe andere Erkenntnisse von der Wissenschaft und von den Herstellern. Wir müssen dann Entscheidungen treffen, welche Infrastruktureinrichtungen, welche Anlagen wir in Deutschland errichten, um uns unabhängiger zu machen, auch was das Thema „alternative Kraftstoffe“ betrifft. Da sind wir gerade in der Diskussion. Zum zweiten Teil Ihrer Frage. Wir haben das Klimakabinett gerade erst gegründet. Wir haben aber letzte Woche vereinbart, dass die Experten dieser nationalen Plattform auch unserem Bereich zuarbeiten, dass wir jetzt erst die Diskussion beginnen, welche Ziele wir im Klimakabinett sek